In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
     |████████████████████████████████| 61 kB 496 kB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

     |████████████████████████████████| 306 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 132 kB 73.0 MB/s 
     |████████████████████████████████| 243 kB 59.7 MB/s 
     |████████████████████████████████| 160 kB 57.8 MB/s 
     |████████████████████████████████| 192 kB 59.9 MB/s 
     |████████████████████████████████| 271 kB 53.1 MB/s 


In [3]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv


--2022-01-01 14:28:17--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364313 (356K) [text/plain]
Saving to: ‘BASA_cricket_train.tsv’

BASA_cricket_train. 100%[===================>] 355.77K  --.-KB/s    in 0.04s   

2022-01-01 14:28:17 (9.88 MB/s) - ‘BASA_cricket_train.tsv’ saved [364313/364313]



In [4]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv

--2022-01-01 14:28:25--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66936 (65K) [text/plain]
Saving to: ‘BASA_cricket_test.tsv’

BASA_cricket_test.t 100%[===================>]  65.37K  --.-KB/s    in 0.01s   

2022-01-01 14:28:26 (5.22 MB/s) - ‘BASA_cricket_test.tsv’ saved [66936/66936]



In [5]:
import pandas as pd

train_df = pd.read_csv("BASA_cricket_train.tsv", sep = "\t")
test_df = pd.read_csv("BASA_cricket_test.tsv", sep = "\t")

In [6]:
train_df.tail(10)

,id,text,class_label
2075,1176,আমাদের ব্যাটসম্যানরা বাজে শটস খেলে আউট হয়েছে। ...,positive
2076,1511,সময় এসেছে রাজাকার পিচ কিউরেটর কে পাছায় লাথি দি...,negative
2077,2463,ক্রিকেটারদের প্রতিযোগিতামূলক ক্রিকেট সম্বন্ধে ...,negative
2078,1038,"দল জেতায় মুশি ক্রেডিট ম্যাসের,দল জেতায় রিয়াদ আ...",negative
2079,2911,তাদের স্পীন এটাক যে আমাদের থেকে ভালো তা কি এই ...,negative
2080,1137,বিদেশী ৫ জন এক ম্যাচে ঠিকই আছে।,neutral
2081,511,দ্রুত তাকে জাতীয়দলে সুযোগ দিয়ে ক্যারিয়ার ধংস্ব...,negative
2082,2611,"ওই নিম্ন মানের পাড়ার ক্রিকেট, ঘরোয়া লীগ, বিপিএ...",negative
2083,2613,নিলয় সবজায়গায় দেখি মাশরাফি নিয়ে বাজে কমেন্ট কর...,negative
2084,908,দ্বিতীয় ইনিংসে লিটন দাসের ট্রিপল সেঞ্চুরি চাই,positive


In [7]:
len(train_df)

2085

In [9]:
train_df["class_label"].value_counts()

negative    1515
positive     376
neutral      194
Name: class_label, dtype: int64

In [10]:
def class_to_numeric(df_column):
  if df_column == "neutral":
    return 0
  elif df_column == "positive":
    return 1
  elif df_column == "negative":
    return 2

In [11]:
train_labels = pd.Series(train_df["class_label"].apply(class_to_numeric))

In [12]:
len(train_labels) ==  len(train_df)

True

In [13]:
train_df.loc[1]

id                                                           955
text           মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...
class_label                                             positive
Name: 1, dtype: object

In [14]:
train_df["labels"] = train_labels
train_df.head()

,id,text,class_label,labels
0,2760,আফতাব আপনি ভালো আছেন?,neutral,0
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive,1
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral,0
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative,2
4,2490,শুভকামনা রাজ্জাক ভাই।,negative,2


In [15]:
test_labels = pd.Series(test_df["class_label"].apply(class_to_numeric))
test_df["labels"] = test_labels
test_df.head()

,id,text,class_label,labels
0,2754,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,negative,2
1,2746,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",negative,2
2,21,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,negative,2
3,2143,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,negative,2
4,426,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,negative,2


In [16]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
train_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 2085
})

In [17]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 380
})